Here is a short tutorial on how to access weights in a network, copy and write values directly

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models 

Let's take the network that was defined in the pytorch tutorial and generate a random classification problem

In [2]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleNet()
optimizer = torch.optim.SGD(model.parameters(), lr=1.)
criterion = nn.CrossEntropyLoss()

## Generating just a single example of data and target
data = torch.randn(1, 1, 32, 32)
target = torch.randint(0, 10, (1,))

### doing three epochs of optimization
for _ in range(3):
    out = model(data)
    loss = criterion(out, target)
    loss.backward()
    optimizer.step()


Now let's have a look at the weights. Weights can be accessed by using the name of the layer, and the weights property. For example 

In [3]:
model.fc1.weight

Parameter containing:
tensor([[-0.0190, -0.0214,  0.0443,  ..., -0.0270,  0.0069, -0.0229],
        [ 0.0389,  0.0178,  0.0239,  ..., -0.0206,  0.0092, -0.0302],
        [ 0.0309, -0.0155,  0.0243,  ..., -0.0073, -0.0308, -0.0098],
        ...,
        [ 0.0239, -0.0097, -0.0216,  ..., -0.0380,  0.0001, -0.0399],
        [-0.0241,  0.0358, -0.0399,  ...,  0.0054, -0.0128,  0.0218],
        [-0.0267, -0.0023,  0.0251,  ...,  0.0267,  0.0235,  0.0068]],
       requires_grad=True)

You can access just the tensor data by doing :

In [4]:
model.fc1.weight.data

tensor([[-0.0190, -0.0214,  0.0443,  ..., -0.0270,  0.0069, -0.0229],
        [ 0.0389,  0.0178,  0.0239,  ..., -0.0206,  0.0092, -0.0302],
        [ 0.0309, -0.0155,  0.0243,  ..., -0.0073, -0.0308, -0.0098],
        ...,
        [ 0.0239, -0.0097, -0.0216,  ..., -0.0380,  0.0001, -0.0399],
        [-0.0241,  0.0358, -0.0399,  ...,  0.0054, -0.0128,  0.0218],
        [-0.0267, -0.0023,  0.0251,  ...,  0.0267,  0.0235,  0.0068]])

In order to copy these values into another place (e.g. to store the full precision version of the weights), use the clone function followed by a copy_ 

In [5]:
weights_fc1_initialvalue = model.fc1.weight.clone()
weights_fc1_initialvalue.copy_(model.fc1.weight.data)

tensor([[-0.0190, -0.0214,  0.0443,  ..., -0.0270,  0.0069, -0.0229],
        [ 0.0389,  0.0178,  0.0239,  ..., -0.0206,  0.0092, -0.0302],
        [ 0.0309, -0.0155,  0.0243,  ..., -0.0073, -0.0308, -0.0098],
        ...,
        [ 0.0239, -0.0097, -0.0216,  ..., -0.0380,  0.0001, -0.0399],
        [-0.0241,  0.0358, -0.0399,  ...,  0.0054, -0.0128,  0.0218],
        [-0.0267, -0.0023,  0.0251,  ...,  0.0267,  0.0235,  0.0068]],
       grad_fn=<CopyBackwards>)

Now let's say you want to modify all the values of the weight by hand, for instance let's replace those weights by their absolute value. This can be done by : 

In [6]:
model.fc1.weight.data.copy_(torch.abs(model.fc1.weight.data))

tensor([[0.0190, 0.0214, 0.0443,  ..., 0.0270, 0.0069, 0.0229],
        [0.0389, 0.0178, 0.0239,  ..., 0.0206, 0.0092, 0.0302],
        [0.0309, 0.0155, 0.0243,  ..., 0.0073, 0.0308, 0.0098],
        ...,
        [0.0239, 0.0097, 0.0216,  ..., 0.0380, 0.0001, 0.0399],
        [0.0241, 0.0358, 0.0399,  ..., 0.0054, 0.0128, 0.0218],
        [0.0267, 0.0023, 0.0251,  ..., 0.0267, 0.0235, 0.0068]])

Let's check that the weights have been modified

In [7]:
model.fc1.weight.data

tensor([[0.0190, 0.0214, 0.0443,  ..., 0.0270, 0.0069, 0.0229],
        [0.0389, 0.0178, 0.0239,  ..., 0.0206, 0.0092, 0.0302],
        [0.0309, 0.0155, 0.0243,  ..., 0.0073, 0.0308, 0.0098],
        ...,
        [0.0239, 0.0097, 0.0216,  ..., 0.0380, 0.0001, 0.0399],
        [0.0241, 0.0358, 0.0399,  ..., 0.0054, 0.0128, 0.0218],
        [0.0267, 0.0023, 0.0251,  ..., 0.0267, 0.0235, 0.0068]])

Now we would like to restore the previous values of the weights. Fortunately we have saved them previously. So we just have to do : 

In [8]:
model.fc1.weight.data.copy_(weights_fc1_initialvalue.data)

tensor([[-0.0190, -0.0214,  0.0443,  ..., -0.0270,  0.0069, -0.0229],
        [ 0.0389,  0.0178,  0.0239,  ..., -0.0206,  0.0092, -0.0302],
        [ 0.0309, -0.0155,  0.0243,  ..., -0.0073, -0.0308, -0.0098],
        ...,
        [ 0.0239, -0.0097, -0.0216,  ..., -0.0380,  0.0001, -0.0399],
        [-0.0241,  0.0358, -0.0399,  ...,  0.0054, -0.0128,  0.0218],
        [-0.0267, -0.0023,  0.0251,  ...,  0.0267,  0.0235,  0.0068]])